In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

In [90]:
import numpy as np
from matplotlib import pyplot as plt

from dl import Model
from dl import Dense
from dl.activations import TanH, ReLU, LeakyReLU
from dl.losses import MAE, MSE
from dl.regularization import Dropout
from dl.optimizers import AdaptiveMomentEstimation, MomentumGradientDescent

In [3]:
m, n = 2000, 10
inputs = np.random.randn(m, n)
outputs = 2 * inputs + 1

In [4]:
# train, val, test split
def train_test_split(inputs, outputs, pourcentage):
    m, _ = inputs.shape
    permutation = np.random.permutation(m)
    m_train = np.ceil(m * pourcentage).astype(int)

    inputs_train = inputs[permutation[:m_train]]
    inputs_test = inputs[permutation[m_train:]]

    outputs_train = outputs[permutation[:m_train]]
    outputs_test = outputs[permutation[m_train:]]

    return inputs_train, outputs_train, inputs_test, outputs_test

inputs_train, outputs_train, inputs_test, outputs_test = train_test_split(inputs, outputs, pourcentage=0.8)
inputs_val, outputs_val, inputs_test, outputs_test = train_test_split(inputs_test, outputs_test, pourcentage=0.8)

print(inputs_train.shape, outputs_train.shape)
print(inputs_test.shape, outputs_test.shape)
print(inputs_val.shape, outputs_val.shape)

(1600, 10) (1600, 10)
(80, 10) (80, 10)
(320, 10) (320, 10)


In [104]:
model = Model([
    Dense(n, 32),
    ReLU(),
    Dropout(keep_prob=0.6),
    Dense(32, 5),
    ReLU(),
    Dropout(keep_prob=0.8),
    Dense(5, 32),
    LeakyReLU(0.3),
    Dropout(keep_prob=0.4),
    Dense(32, n)
])

In [105]:
n_epochs = 500
learning_rate = 0.01
batch_size = 128
beta1 = 0.9
beta2 = 0.99

In [106]:
loss = MSE()
# loss = MAE()

In [107]:
optimizer = AdaptiveMomentEstimation(learning_rate, batch_size, beta1, beta2)
# optimizer = MomentumGradientDescent(learning_rate, batch_size, beta1)

In [108]:
model.compile(loss, optimizer)

In [109]:
history = model.optimize(inputs_train, outputs_train, n_epochs, verbose=True, inputs_val=inputs_val, outputs_val=outputs_val)

[1/500]: loss = 26063.904063288323
[2/500]: loss = 2716.4197025212534
[3/500]: loss = 436.6937085761498
[4/500]: loss = 176.74825967282584
[5/500]: loss = 122.78082854429222
[6/500]: loss = 98.54273007109929
[7/500]: loss = 84.54565983985952
[8/500]: loss = 76.39470386375854
[9/500]: loss = 66.83841632026976
[10/500]: loss = 58.59498237676705
[11/500]: loss = 53.70006453769306
[12/500]: loss = 50.62405381335565
[13/500]: loss = 43.449607585778615
[14/500]: loss = 35.25583112318268
[15/500]: loss = 31.551700246178303
[16/500]: loss = 29.298335710837033
[17/500]: loss = 26.03061012488138
[18/500]: loss = 24.42536368060509
[19/500]: loss = 22.30038081837516
[20/500]: loss = 19.71577160800259
[21/500]: loss = 16.140563753590868
[22/500]: loss = 15.01841725383492
[23/500]: loss = 13.682767732091827
[24/500]: loss = 12.489440361667029
[25/500]: loss = 11.271180733567617
[26/500]: loss = 10.277731715178142
[27/500]: loss = 9.247722750320525
[28/500]: loss = 8.517366433870171
[29/500]: loss = 

In [ ]:
X = list(range(len(history["train_losses"])))

fig, axs = plt.subplots(2, sharex=True)
fig.suptitle('Training & Validation Loss')

axs[0].plot(X, history["train_losses"])
axs[0].set_xlabel("Epoch")
axs[0].set_ylabel("Training Loss")

axs[1].plot(X, history["val_losses"])
axs[1].set_xlabel("Epoch")
axs[1].set_ylabel("Validation Loss")

plt.show()

In [112]:
predictions = model(inputs_val, is_optimizing=False)
l = loss.forward(predictions, outputs_val)

print(l)

13.396101469510691


In [113]:
predictions = model(inputs_test, is_optimizing=False)
l = loss.forward(predictions, outputs_test)

print(l)

14.490783399175523
